In [12]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Add

In [13]:
def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image)
    image = tf.image.resize(image, (94, 94))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    image = tf.expand_dims(image, axis=0)

    return image

In [20]:
input_shape = (94, 94, 3)
latent_dim = 32


class Encoder(Model):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=input_shape),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((3, 3), padding='same'),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((3, 3), padding='same'),
            layers.Flatten()
        ])
        self.mean = layers.Dense(latent_dim)
        self.log_var = layers.Dense(latent_dim)
    
    def call(self, X):
        X = self.encoder(X)
        return self.mean(X), self.log_var(X)


class Decoder(Model):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(1, latent_dim)),
            layers.Dense(7 * 7 * 64, activation='relu'),
            layers.Reshape((7, 7, 64)),
            layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2,2)),
            layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2,2)),
            layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')
        ])
    
    def call(self, X):
        return self.decoder(X)


class VAE(Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def call(self, X):
        mu, sigma = self.encoder(X)
        epsilon = tf.random.normal(tf.shape(mu))
        z = tf.expand_dims(mu + epsilon * sigma, axis=0)
        X_reconstructed = self.decoder(z)

        return X_reconstructed, mu, sigma
        

In [21]:
image = load_image('../data/2_2000.jpg')

latent_dim = 32

model = VAE(latent_dim)
model(image)

(<tf.Tensor: shape=(1, 28, 28, 1), dtype=float32, numpy=
 array([[[[0.49968022],
          [0.49940482],
          [0.50054723],
          [0.50120485],
          [0.5005795 ],
          [0.5007196 ],
          [0.50029254],
          [0.50014144],
          [0.5008166 ],
          [0.5023732 ],
          [0.50177   ],
          [0.5007951 ],
          [0.4998943 ],
          [0.49880525],
          [0.49973008],
          [0.5005088 ],
          [0.5009393 ],
          [0.500862  ],
          [0.50080574],
          [0.50142354],
          [0.5004782 ],
          [0.5014942 ],
          [0.50185496],
          [0.5007152 ],
          [0.50140464],
          [0.502011  ],
          [0.502085  ],
          [0.501421  ]],
 
         [[0.4992664 ],
          [0.50085676],
          [0.5015767 ],
          [0.5012068 ],
          [0.50146246],
          [0.5014783 ],
          [0.5006118 ],
          [0.50093836],
          [0.5025303 ],
          [0.5046315 ],
          [0.5035983 ],
    

In [18]:
load_image('../data/110_2000.jpg')

<tf.Tensor: shape=(1, 94, 94, 3), dtype=float32, numpy=
array([[[[0.9953814 , 0.9868372 , 0.9867435 ],
         [0.9752691 , 0.9777532 , 0.9670757 ],
         [0.9098429 , 0.9519794 , 0.9121689 ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[0.96419775, 0.94559556, 0.95154047],
         [0.9538631 , 0.9662574 , 0.9373027 ],
         [0.82508534, 0.8817711 , 0.7989182 ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[0.9882415 , 0.98139936, 0.98551965],
         [0.8733379 , 0.8894521 , 0.84434396],
         [0.80709445, 0.86934   , 0.73895663],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        ...,

        [[0.9536312 , 0.9976117 , 0.

In [24]:
tensor = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
tf.split(tensor, num_or_size_splits=2)

[<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 2.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3., 4.]], dtype=float32)>]